In [1]:
import torch as T
import torch.nn as nn
from sklearn.feature_extraction import image
from sklearn.metrics import confusion_matrix
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from PIL import Image
from skimage import io
import torch.nn.functional as F

In [2]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=2):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(p=0.1),
            nn.Conv2d(64, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(p=0.2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(p=0.3),
           
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512 * 9 * 9, 1000),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1000, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, num_classes),
        )
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.shape[0],-1)
        x = self.classifier(x)
        return x

In [3]:
def bayesian_inference(H, E) :
    # P(A|B) = (P(B|A)*P(A))/P(B)
    # P(A|B) Probabilité à posteriori que l'hypothèse A0 soit vraie (ce que l'on cherche à calculer)
    # P(B|A) Probabilité que l'hypothèse soit vraie, selon le résultat d'un évenement (paramètre E pour évenement)
    # P(A) = Probabilité à prioiri que l'hypothèse A0 est vraie (paramètre H pour hypothèse)
    # P(B) = Probabilité de l'évenement, indépendament de l'hypothèse
         # En supposant que les seuls hypothèse sont A0 et A1 :
         #P(B) = P(B|A0)*P(A0|B) + P(B|A1)
        
    # On ne fait pas très confiance à la preuve 
    confiance = 0.5
    E = E*confiance + (1-confiance)/2
    
    
    return (H * E) / (H * E + (1-H)*(1-E))
        
 

In [4]:
# load the trained model
net = T.load("trained_net")
net.to('cpu')
net.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.1, inplace=False)
    (4): Conv2d(64, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.2, inplace=False)
    (8): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Dropout(p=0.3, inplace=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=41472, out_features=1000, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=1000, out_features=1000, bias=True)
    (5): ReLU(

In [5]:
def predictOneImage(path,img_name,size,number_patches,threshold):
    
    predicted_list = []
    
    # chargement de l'image
    img = io.imread(path+"/"+img_name)
    
    #extraction des patchs
    patches = image.extract_patches_2d(img, (size,size),max_patches=number_patches)
    transform = transforms.Compose([transforms.ToTensor()])
    
    # pour chacun des patch
    for i in range(len(patches)):
        # on applique les mêmes transformations et conditions que pour les images du training
        patches[i] = patches[i].astype('uint16')
        m = np.mean(patches[i])
        if (m>threshold) :
            patch = Image.fromarray(patches[i])
            patch = patch.convert(mode='RGB')
            image_tensor = transform(patch)
            image_tensor = image_tensor.unsqueeze_(0)
            
            # on donne le patch à notre trained model
            output = net(image_tensor)
            
            # on récupère la probabilité que le patch soit issu d'une image de mitochondrie segmentée, 
            #selon l'algorithme
            output = output[0].tolist()
            confidence = F.softmax(T.tensor([[output[0],output[1]]],dtype=T.float),dim=1).tolist()[0][0]
            predicted_list.append(confidence)
                    
        return predicted_list

In [41]:
def predictOne_consistent(path,img_name,size,number_patches,threshold,repeats):
    
    # vu que les patchs sont piochés de manière aléatoire à chaque fois, 
    # j'ai trouvé qu'on obtenait des résultats plus fiables en faisant une moyenne
    # des prédictions sur plusieurs essais sur la même image
    predicted_list = []
    healthy_patches = 0
    fragmented_patches = 0
    for i in range(repeats):
        predicted_list += predictOneImage(path,img_name,size,number_patches,threshold)
    
    
    if len(predicted_list) == 0 :
        print("No prediction can be made")
        return -1,0,0
    consensus_prediction = 0.5
    for prediction in predicted_list :
        if prediction < 0.5 :
            healthy_patches += 1
        else :
            fragmented_patches += 1
        consensus_prediction = bayesian_inference(consensus_prediction, prediction)
        
        
    if consensus_prediction < 0.5:
        print("prediction : HEALTHY (%s)"%(1-consensus_prediction))
    else :
        print("prediction : FRAGMENTED (%s)" %(consensus_prediction))
    return consensus_prediction, healthy_patches, fragmented_patches
    
    

In [42]:
def predictMultiples(path,size,number_patches,threshold,repeats):
    csv = ""
    healthy = 0
    fragmented = 0
    errors = 0
    healthy_patches = 0
    fragmented_patches = 0
    consensus_prediction = 0.5    
    # pour chaque image du répertoire
    for img_name in listdir(path) :
        res = predictOne_consistent(path,img_name,size,number_patches,threshold,repeats)
        healthy_patches += res[1]
        fragmented_patches += res[2]
        
        """
        # création d'un histogramme
        x = np.array(["Healthy", "Fragmented"])
        if(res == -1) :
            y = np.array([0,0])
        else :
            y = np.array([1-res,res])
        fig, ax = plt.subplots()
        barp = plt.bar(x,y, color=['green', 'red'])
        plt.show()
        # on ouvre et affiche l'image
        im = Image.open(path+"/"+img_name)
        im = im.resize((488,360), Image.NEAREST)
        display(im)
        """
    
        csv += img_name + ";" #Les noms des images contiennent des ",", ne pas en utiliser comme séparateur
        if res[0] == -1 :
            csv += "none;none \n"
            errors += 1
        else :
            # Inférence Bayésienne :
            consensus_prediction = bayesian_inference(consensus_prediction, res[0])
            #(consensus_prediction * res) / (consensus_prediction * res + (1-consensus_prediction)*(1-res))    
            if res[0] < 0.5 :
                csv += str(res[0]) + ";healthy"
                healthy += 1
            else :
                csv += str(res[0]) + ";fragmented"
                fragmented += 1
            csv += ";"+ str(res[1])+ ";" + str(res[2])+ "\n" 
        print("============================================================")

    head = "image;P(fragmented);Prediction;Healthy patches;Fragmented patches \n"
    head += "tissu;%s;" %(consensus_prediction)
    if(consensus_prediction > 0.5) :
        head += "fragmented;"
    else :
        head += "healthy;"
    head += str(healthy_patches) + ";" + str(fragmented_patches) + "\n"
    csv = head + csv
    print(csv)
    print(f"il y a eu une prédiction de {healthy} healthy et {fragmented} fragmented sur {healthy + fragmented + errors} images traitées")
    print("confiance : %s" %(consensus_prediction))
    if(errors >0) :
        print("Il y a %s images intraitables" %(errors))
        
        
    fic=open("experimental_result.csv","w")
    fic.write(csv)
    fic.close()

In [58]:
path = "./Data/Images-Cytation/Experiment2_201210/Bad/Prediction"
path = "./Data/All_bad"
size = 320
number_patches = 10
threshold = 3
repeats = 10

predictMultiples(path,size,number_patches,threshold,repeats)

prediction : FRAGMENTED (0.844276849177525)
prediction : FRAGMENTED (0.9980842569329714)
prediction : HEALTHY (0.561969202460898)
prediction : FRAGMENTED (0.9762515051427766)
prediction : FRAGMENTED (0.8937782884156191)
prediction : HEALTHY (0.5296480110280646)
prediction : FRAGMENTED (0.9745203550504405)
prediction : FRAGMENTED (0.9904944499702729)
prediction : HEALTHY (0.9973021601391501)
No prediction can be made
prediction : FRAGMENTED (0.7714375051943824)
prediction : HEALTHY (0.5883966294713641)
prediction : FRAGMENTED (0.7119072489870701)
prediction : FRAGMENTED (0.9651641482193829)
prediction : FRAGMENTED (0.9650386282383938)
prediction : HEALTHY (0.8355000016313499)
prediction : HEALTHY (0.8489827681453398)
prediction : FRAGMENTED (0.6100608958555607)
prediction : FRAGMENTED (0.6069160997867584)
prediction : HEALTHY (0.7868416707665136)
prediction : FRAGMENTED (0.9693976815776194)
prediction : FRAGMENTED (0.5453206598758698)
prediction : FRAGMENTED (0.6899114694888477)
predict

prediction : FRAGMENTED (0.9979786402155315)
prediction : FRAGMENTED (0.9417345420144729)
No prediction can be made
prediction : HEALTHY (0.8611454083358465)
prediction : HEALTHY (0.8353218350968429)
prediction : FRAGMENTED (0.9927977585523444)
prediction : FRAGMENTED (0.7844172987407213)
prediction : HEALTHY (0.6361943072921704)
prediction : FRAGMENTED (0.9940021126511228)
prediction : FRAGMENTED (0.9006415524591455)
prediction : FRAGMENTED (0.6000262796878815)
prediction : FRAGMENTED (0.5247275829315186)
prediction : FRAGMENTED (0.9728884700653697)
No prediction can be made
prediction : FRAGMENTED (0.9298488486211453)
No prediction can be made
No prediction can be made
prediction : HEALTHY (0.9961836896422472)
prediction : FRAGMENTED (0.68535643268399)
prediction : FRAGMENTED (0.7492785190082775)
prediction : FRAGMENTED (0.83340195260074)
prediction : FRAGMENTED (0.6717544520398806)
prediction : FRAGMENTED (0.9533533536213696)
No prediction can be made
prediction : HEALTHY (0.9704140

prediction : HEALTHY (0.9914516893725854)
prediction : FRAGMENTED (0.6349940844555625)
prediction : FRAGMENTED (0.7187860619189473)
prediction : FRAGMENTED (0.8694110920614037)
prediction : HEALTHY (0.9935483191885747)
prediction : FRAGMENTED (0.9432076275751513)
prediction : FRAGMENTED (0.9898000758898338)
prediction : FRAGMENTED (0.9503379039667612)
prediction : HEALTHY (0.9893175008368479)
prediction : FRAGMENTED (0.9709438400551097)
prediction : FRAGMENTED (0.5579161047935486)
prediction : HEALTHY (0.9744049088865192)
prediction : FRAGMENTED (0.8779719282769824)
prediction : FRAGMENTED (0.9605160497288617)
prediction : FRAGMENTED (0.9985458527498444)
prediction : HEALTHY (0.8478248189244456)
prediction : FRAGMENTED (0.9933828800386943)
prediction : FRAGMENTED (0.9954704185123583)
prediction : FRAGMENTED (0.8244496996746512)
prediction : FRAGMENTED (0.5161080811216994)
prediction : FRAGMENTED (0.743529621018828)
prediction : HEALTHY (0.5076104649468411)
prediction : HEALTHY (0.96127

prediction : FRAGMENTED (0.8092034542513733)
prediction : FRAGMENTED (0.9980933614290789)
prediction : FRAGMENTED (0.8442656055057559)
image;P(fragmented);Prediction;Healthy patches;Fragmented patches 
tissu;0.9999999999999744;fragmented;531;821
F8--1-1-1-Tsf[GFP 469,525]-001.tif;0.844276849177525;fragmented;2;5
E3-02-2-3-GFP-001.tif;0.9980842569329714;fragmented;0;10
C6--1-1-1-Tsf[GFP 469,525]-001.tif;0.438030797539102;healthy;3;2
H5--2-1-1-Tsf[GFP 469,525]-001.tif;0.9762515051427766;fragmented;0;8
G9--2-1-1-Tsf[GFP 469,525]-001.tif;0.8937782884156191;fragmented;1;7
H11--1-1-1-Tsf[GFP 469,525]-001.tif;0.4703519889719354;healthy;1;2
H7--2-1-1-Tsf[GFP 469,525]-001.tif;0.9745203550504405;fragmented;2;7
D6--1-1-1-Tsf[GFP 469,525]-001.tif;0.9904944499702729;fragmented;0;10
C7--2-1-1-Tsf[GFP 469,525]-001.tif;0.002697839860849873;healthy;9;1
H3--1-1-1-Tsf[GFP 469,525]-001.tif;none;none 
F5--2-1-1-Tsf[GFP 469,525]-001.tif;0.7714375051943824;fragmented;0;2
H3-02-2-1-GFP-001.tif;0.4116033705286